# CTMS Model - Automated Configuration Search

This notebook automatically searches for the best configuration by:
1. Testing different numbers of CN samples for training
2. Comparing without personalization (fixed weights) vs with personalization (adaptive weights)
3. Saving the best configurations

**Goal**: CN should have significantly fewer anomalies than CI

## 1. Setup and Imports

In [7]:
import sys
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from collections import defaultdict
from datetime import datetime
import copy

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Set device - utilize RTX 4090
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("Using CPU")

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Import CTMS modules
from ctms_model_gpu import CTMSModelGPU

print("✓ CTMS modules imported successfully")


# Data paths - UPDATE THESE
SEQUENCE_FILE = '../sample_data/sequences.jsonl'
LABEL_FILE = '../sample_data/subjects.json'

# Search configuration
SEARCH_CONFIG = {
    'cn_train_ratios': [0.3, 0.5, 0.7, 0.8],  # Test different amounts of CN for training
    'use_ci_in_train': [False, True],  # Try with/without CI in training
    'ci_train_ratio': 0.3,  # If using CI in training, how much
    'epochs_per_config': 30,  # Epochs for each configuration
    'batch_size': 64,  # Larger batch for 4090
    'learning_rate': 2e-4,
}

# Weight configurations
WEIGHT_CONFIGS = {
    'without_personalization': [
        [0.25, 0.25, 0.25, 0.25],  # Equal weights
        [0.4, 0.3, 0.2, 0.1],      # Emphasize circadian
        [0.3, 0.4, 0.2, 0.1],      # Emphasize task
        [0.3, 0.3, 0.3, 0.1],      # De-emphasize social
        [0.35, 0.35, 0.2, 0.1],    # Balanced circadian+task
    ],
    'with_personalization': True  # Will be adaptive per person
}

print("Configuration loaded")
print(f"Will test {len(SEARCH_CONFIG['cn_train_ratios']) * len(SEARCH_CONFIG['use_ci_in_train'])} training splits")
print(f"Will test {len(WEIGHT_CONFIGS['without_personalization'])} fixed weight configs")
print(f"Plus adaptive weight personalization")


def load_data(sequence_file, label_file):
    """Load sequences and labels"""
    # Load sequences
    sequences = defaultdict(list)
    with open(sequence_file, 'r') as f:
        for line in f:
            data = json.loads(line.strip())
            sequences[data['anon_id']].append(data)
    
    # Load labels
    with open(label_file, 'r') as f:
        label_data = json.load(f)
        if isinstance(label_data, list):
            labels = {item['anon_id']: item for item in label_data}
        else:
            labels = label_data
    
    return sequences, labels

sequences, labels = load_data(SEQUENCE_FILE, LABEL_FILE)

# Separate CN and CI
cn_ids = [aid for aid, label in labels.items() if label['label'] == 'CN']
ci_ids = [aid for aid, label in labels.items() if label['label'] == 'CI']

print(f"✓ Loaded data")
print(f"  CN participants: {len(cn_ids)}")
print(f"  CI participants: {len(ci_ids)}")
print(f"  Total sequences: {sum(len(s) for s in sequences.values())}")

Using GPU: NVIDIA GeForce RTX 4090
GPU Memory: 25.36 GB
✓ CTMS modules imported successfully
Configuration loaded
Will test 8 training splits
Will test 5 fixed weight configs
Plus adaptive weight personalization
✓ Loaded data
  CN participants: 26
  CI participants: 42
  Total sequences: 128


In [8]:
# ----- Option B: 构建df_all（使用 CTMSModelGPU，从 sequences 推断 cdi/tir/me/sws）
# 说明：可能较慢；默认只跑前 N 个参与者用于 smoke test，可修改 MAX_SUBJECTS=None 遍历全部
from tqdm.notebook import tqdm
import torch

if 'df_all' not in globals():
    print("[INFO] 构建 df_all：使用 CTMSModelGPU 从原始 sequences 中提取行为度量（默认前 50 个 subject）。")
    MAX_SUBJECTS = None  # 设置为 None 则遍历全部
    subj_ids = list(sequences.keys())
    if MAX_SUBJECTS is not None:
        subj_ids = subj_ids[:MAX_SUBJECTS]

    # 初始化模型（与 notebook 其它部分一致的超参）
    model = CTMSModelGPU(d_model=128, num_activities=21, num_task_templates=20, use_fast_similarity=True)
    model.eval()
    model.to(device)

    rows = []
    for aid in tqdm(subj_ids, desc="subjects"):
        recs = sequences.get(aid, [])
        activity_ids = []
        timestamps = []
        for rec in recs:
            seq = rec.get('sequence', []) or rec.get('seq', [])
            for ev in seq:
                # 支持多种可能的字段名
                act_field = None
                for k in ('action_id','activity_id','activity','action'):
                    if k in ev:
                        act_field = ev.get(k)
                        break
                ts_field = None
                for k in ('ts','timestamp','time'):
                    if k in ev:
                        ts_field = ev.get(k)
                        break
                if act_field is None:
                    continue
                try:
                    activity_ids.append(int(act_field))
                except Exception:
                    # 若无法转为 int，跳过
                    continue
                timestamps.append(int(ts_field) if ts_field is not None else 0)

        if len(activity_ids) == 0:
            # 没有事件，跳过
            print(f"[WARN] subject {aid} has no events; skipping")
            continue

        act_tensor = torch.tensor([activity_ids], dtype=torch.long, device=device)  # [1, seq_len]
        ts_tensor  = torch.tensor([timestamps], dtype=torch.long, device=device)

        try:
            with torch.no_grad():
                outputs = model.forward(act_tensor, ts_tensor, return_encodings_only=False)
        except Exception as e:
            print(f"[ERROR] model.forward failed for subject {aid}: {e}")
            continue

        # 按照 ctms_model.py 的规范直接读取标准字段
        try:
            cdi = float(outputs['cdi'].detach().cpu().item()) if 'cdi' in outputs else float(outputs.get('CDI', 0.0))
        except Exception:
            cdi = float(outputs.get('cdi', 0.0))
        try:
            tir = float(outputs['tir'].detach().cpu().item()) if 'tir' in outputs else float(outputs.get('TIR', 0.0))
        except Exception:
            tir = float(outputs.get('tir', 0.0))
        try:
            me  = float(outputs['me'].detach().cpu().item()) if 'me' in outputs else float(outputs.get('ME', 0.0))
        except Exception:
            me = float(outputs.get('me', 0.0))
        try:
            sws = float(outputs['sws'].detach().cpu().item()) if 'sws' in outputs else float(outputs.get('SWS', 0.0))
        except Exception:
            sws = float(outputs.get('sws', 0.0))

        rows.append({
            'anon_id': aid,
            'encoding': [cdi, tir, me, sws],
            'circadian': float(cdi),
            'task': float(tir),
            'movement': float(me),
            'social': float(sws),
            'label': labels.get(aid, {}).get('label', 'CN'),
            'label_binary': 0 if labels.get(aid, {}).get('label','CN')=='CN' else 1
        })

    df_all = pd.DataFrame(rows)
    print(f"[INFO] df_all 构建完成，n={len(df_all)}")
else:
    print("[INFO] df_all 已存在，跳过构建")


[INFO] df_all 已存在，跳过构建


In [9]:
# ============================================================
# 固定权重 vs 个性化权重：自动选最优配置（支持 oracle / zscore 两种策略）
# ============================================================
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from itertools import product

# ------------- 基础工具 -------------
def split_train_test_cn(df, train_ratio=0.5, seed=123):
    rng = np.random.default_rng(seed)
    cn = df[df['label']=="CN"].sample(frac=1.0, random_state=seed).reset_index(drop=True)
    ci = df[df['label']=="CI"].reset_index(drop=True)
    n_train = max(1, int(len(cn)*train_ratio))
    train_cn = cn.iloc[:n_train].copy()
    test = pd.concat([cn.iloc[n_train:], ci], ignore_index=True)
    return train_cn, test

def center_std(train_cn):
    c = np.array([train_cn['circadian'].mean(),
                  train_cn['task'].mean(),
                  train_cn['movement'].mean(),
                  train_cn['social'].mean()])
    s = np.array([train_cn['circadian'].std(ddof=1),
                  train_cn['task'].std(ddof=1),
                  train_cn['movement'].std(ddof=1),
                  train_cn['social'].std(ddof=1)])
    s[~np.isfinite(s)] = 1.0
    s[s==0] = 1.0
    return c, s

def distances(df, center, std, w):
    z = (np.stack(df['encoding'].to_list(), axis=0) - center) / std
    w = np.asarray(w, float)
    return np.sqrt((w*(z**2)).sum(axis=1))

def scan_threshold(d, y, n=200):
    d = np.asarray(d); y = np.asarray(y).astype(int)
    ths = np.linspace(d.min(), d.max(), n)
    best = {"threshold": None, "f1": -1, "acc": None,
            "prec": None, "rec": None}
    for th in ths:
        yp = (d > th).astype(int)
        if len(np.unique(yp)) < 2:  # 全同
            continue
        f1 = f1_score(y, yp)
        acc = accuracy_score(y, yp)
        prec = precision_score(y, yp, zero_division=0)
        rec  = recall_score(y, yp, zero_division=0)
        if f1 > best["f1"]:
            best.update({"threshold": th, "f1": f1, "acc": acc, "prec": prec, "rec": rec})
    return best

def mannwhitney_p(cn_d, ci_d):
    try:
        _, p = stats.mannwhitneyu(cn_d, ci_d, alternative="two-sided")
    except Exception:
        p = np.nan
    return p

# ------------- 权重候选集合 -------------
def make_weight_candidates():
    # 预设若干“极端/偏置”组合 LLM 推荐
    presets = [
        ("Equal",                [0.25, 0.25, 0.25, 0.25]),
        ("Circadian++",          [0.50, 0.35, 0.10, 0.05]),
        ("Task++",               [0.35, 0.50, 0.10, 0.05]),
        ("Circadian-dominant",   [0.70, 0.20, 0.05, 0.05]),
        ("Task-dominant",        [0.20, 0.70, 0.05, 0.05]),
        ("Movement++",           [0.10, 0.10, 0.70, 0.10]),
        ("Social++",             [0.10, 0.10, 0.10, 0.70]),
        ("CT+++",                [0.50, 0.40, 0.05, 0.05]),
        ("CT-Balanced",          [0.45, 0.45, 0.05, 0.05]),
    ]
    # 网格细化（强化 C/T，M/S 均分剩余）
    name2w = {n: np.array(w, float) for n, w in presets}
    for c, t in product(np.arange(0.45, 0.81, 0.05), np.arange(0.35, 0.71, 0.05)):
        rest = 1.0 - (c + t)
        if rest < 0.10:  # 给 M+S 至少 0.10
            continue
        m = s = rest/2
        if m < 0.05 or s < 0.05:
            continue
        name2w[f"Grid_C{c:.2f}_T{t:.2f}"] = np.array([c, t, m, s], float)
    return name2w

# ------------- without personalization：全局最优 -------------
def evaluate_global(df, train_ratio=0.5, seed=123, candidates=None):
    if candidates is None:
      candidates = make_weight_candidates()
    train_cn, test = split_train_test_cn(df, train_ratio, seed)
    c, s = center_std(train_cn)
    y = test['label_binary'].values

    rows = []
    context = {}
    for name, w in candidates.items():
        d = distances(test, c, s, w)
        best = scan_threshold(d, y)
        cn_d = d[test['label'].values=='CN']; ci_d = d[test['label'].values=='CI']
        rows.append({
            "weights_name": name, "weights": w.tolist(),
            "threshold": best["threshold"],
            "f1": best["f1"], "acc": best["acc"],
            "prec": best["prec"], "rec": best["rec"],
            "p_value": mannwhitney_p(cn_d, ci_d),
            "sep": float(ci_d.mean() - cn_d.mean())
        })
        context[name] = {"d": d, "y": y, "center": c, "std": s, "threshold": best["threshold"]}
    res = pd.DataFrame(rows).sort_values(["p_value","sep"], ascending=[True, False]).reset_index(drop=True)
    best_row = res.iloc[0]
    return {"results": res, "best": best_row, "context": context, "train_cn": train_cn, "test": test}

# ------------- with personalization：为每个人挑权重 -------------
def evaluate_personalized(df, train_ratio=0.5, seed=123, candidates=None,
                          method="fixed_fpr", alpha=0.975,
                          global_fallback=None):
    """
    method:
      - 'fixed_fpr'：无标签。每个权重用训练CN的分位数阈值；为每个样本选 margin 最大的权重；
                     预测= (max_margin>0)。支持回退到全局最优（可选）。
      - 'oracle'   ：上限评估（与之前一致）。
    global_fallback: 传入一个 dict，形如
        {"weights": np.array([...]), "threshold": float, "center": c, "std": s}
      可用 evaluate_global(...) 的 best 配置构建；为空则不回退。
    """
    if candidates is None:
        candidates = make_weight_candidates()

    train_cn, test = split_train_test_cn(df, train_ratio, seed)
    c, s = center_std(train_cn)
    y = test['label_binary'].values
    N = len(test)

    # 预计算每个权重的距离 & 阈值
    per_weight = {}
    for name, w in candidates.items():
        d_test  = distances(test, c, s, w)
        d_train = distances(train_cn, c, s, w)
        if method == "oracle":
            th = scan_threshold(d_test, y)["threshold"]
        else:  # fixed_fpr
            th = np.quantile(d_train, alpha)  # 控 CN 的假阳率
        per_weight[name] = {"w": w, "d_test": d_test, "th": th}

    # （可选）全局回退
    gf = None
    if global_fallback is not None:
        w_g  = np.array(global_fallback["weights"], float)
        th_g = float(global_fallback["threshold"])
        c_g  = np.array(global_fallback["center"], float)
        s_g  = np.array(global_fallback["std"], float)
        d_g  = distances(test, c_g, s_g, w_g)
        gf = {"d": d_g, "th": th_g}

    # 逐人选择权重：最大 margin
    chosen = []
    y_pred = np.zeros(N, dtype=int)
    margins = np.zeros(N, dtype=float)
    for i in range(N):
        best_name, best_margin, best_pred = None, -np.inf, 0
        for name, obj in per_weight.items():
            d_i  = obj["d_test"][i]
            m_i  = d_i - obj["th"]     # >0 说明“超阈值”
            pred = int(m_i > 0.0)
            if m_i > best_margin:
                best_name, best_margin, best_pred = name, m_i, pred

        # 回退逻辑（可选）：若所有 margin<=0 且全局能给出正margin，就用全局
        if gf is not None and best_margin <= 0.0:
            m_g = gf["d"][i] - gf["th"]
            if m_g > best_margin:
                best_name, best_margin, best_pred = "GLOBAL_FALLBACK", m_g, int(m_g > 0.0)

        chosen.append(best_name)
        y_pred[i] = best_pred
        margins[i] = best_margin

    # 汇总指标
    from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
    f1  = f1_score(y, y_pred)
    acc = accuracy_score(y, y_pred)
    prec = precision_score(y, y_pred, zero_division=0)
    rec  = recall_score(y, y_pred, zero_division=0)

    # 统计显著性（用被选权重的距离）
    d_chosen = np.array([
        (gf["d"][i] if chosen[i]=="GLOBAL_FALLBACK" else per_weight[chosen[i]]["d_test"][i])
        for i in range(N)
    ])
    cn_mask, ci_mask = (y_pred==0), (y_pred==1)
    try:
        from scipy import stats
        p_val = stats.mannwhitneyu(d_chosen[cn_mask], d_chosen[ci_mask], alternative="two-sided").pvalue \
                if cn_mask.any() and ci_mask.any() else np.nan
    except Exception:
        p_val = np.nan

    summary = {
        "method": method,
        "alpha": alpha,
        "f1": f1, "acc": acc, "prec": prec, "rec": rec,
        "p_value": p_val,
        "weights_top_counts": pd.Series(chosen).value_counts().to_dict()
    }
    detail = pd.DataFrame({
        "anon_id": test["anon_id"].values if "anon_id" in test.columns else np.arange(N),
        "true": y, "pred": y_pred,
        "chosen_weight": chosen, "margin": margins
    })
    return summary, detail, {"train_cn": train_cn, "test": test, "per_weight": per_weight, "global_fallback": gf}

# ===================== 一键运行 =====================
# 1) 全局固定权重（without personalization）
global_out = evaluate_global(df_all, train_ratio=0.5, seed=123)
print("\n=== WITHOUT personalization（全局最优）===")
print(global_out["results"].head(10))  # Top10
print("\nBEST (global):")
print(global_out["best"])

# 2) 个性化权重（with personalization）
# 2.1 无泄露（zscore）
pers_z_summary, pers_z_detail, _ = evaluate_personalized(df_all, train_ratio=0.5, seed=123, method="fixed_fpr", alpha=0.95)
print("\n=== WITH personalization (fixed_fpr, 无标签) ===")
print(pers_z_summary)
# 可保存
pers_z_detail.to_csv("with_personalization_zscore_detail.csv", index=False)

# 2.2 上限评估（oracle）
pers_orc_summary, pers_orc_detail, _ = evaluate_personalized(df_all, train_ratio=0.5, seed=123, method="oracle")
print("\n=== WITH personalization (oracle, 上限) ===")
print(pers_orc_summary)
# 可保存
pers_orc_detail.to_csv("with_personalization_oracle_detail.csv", index=False)

# 3) 也把全局结果保存一下
global_out["results"].to_csv("without_personalization_global_scan.csv", index=False)
print("\n✓ 保存文件：")
print(" - without_personalization_global_scan.csv")
print(" - with_personalization_zscore_detail.csv")
print(" - with_personalization_oracle_detail.csv")

# ============================================================
# 保存 best without / best with（含 Train/Test 划分与逐人权重）
# 替换你之前的保存 JSON 代码段
# ============================================================
import json
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, matthews_corrcoef
# ===== 放在保存 JSON 段的最上面：通用“去NumPy化”工具 =====

import os, json, numpy as np, pandas as pd

def to_builtin(o):
    if isinstance(o, dict):
        return {to_builtin(k): to_builtin(v) for k, v in o.items()}
    if isinstance(o, (list, tuple, set)):
        return [to_builtin(x) for x in o]
    if isinstance(o, (np.integer,)):   return int(o)
    if isinstance(o, (np.floating,)):  return float(o)
    if isinstance(o, (np.bool_,)):     return bool(o)
    if isinstance(o, (np.ndarray,)):   return o.tolist()
    if isinstance(o, pd.Series):       return o.tolist()
    if isinstance(o, pd.DataFrame):    return o.to_dict(orient="records")
    return o

def json_dump_safe(obj, path):
    with open(path, "w") as f:
        json.dump(to_builtin(obj), f, indent=2)

# ---- 工具：富指标 ----
def compute_metrics_rich(y_true, y_pred):
    y_true = np.asarray(y_true).astype(int)
    y_pred = np.asarray(y_pred).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    acc = (tp + tn) / max(1, (tp + tn + fp + fn))
    prec = tp / max(1, (tp + fp))
    rec = tp / max(1, (tp + fn))                     # sensitivity / recall
    spec = tn / max(1, (tn + fp))                    # specificity
    f1 = (2*prec*rec / (prec + rec)) if (prec + rec) > 0 else 0.0
    bal_acc = 0.5 * (rec + spec)
    try:
        mcc = matthews_corrcoef(y_true, y_pred)
        if not np.isfinite(mcc): mcc = 0.0
    except Exception:
        mcc = 0.0
    return {
        "accuracy": float(acc),
        "precision": float(prec),
        "recall_sensitivity": float(rec),
        "specificity": float(spec),
        "f1": float(f1),
        "balanced_accuracy": float(bal_acc),
        "mcc": float(mcc),
        "confusion_matrix": {"tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)},
        "fpr": float(1.0 - spec),
        "fnr": float(1.0 - rec),
        "tpr": float(rec),
        "tnr": float(spec),
    }

# ---------------- best WITHOUT ----------------
_best_row = global_out["best"]
_best_name = _best_row["weights_name"]
_ctx = global_out["context"][_best_name]

y_true_wo = _ctx["y"]
dists_wo  = _ctx["d"]
th_wo     = float(_ctx["threshold"])
y_pred_wo = (dists_wo > th_wo).astype(int)
metrics_wo = compute_metrics_rich(y_true_wo, y_pred_wo)

# 记录划分（train_cn/test）
train_cn_df = global_out["train_cn"]
test_df     = global_out["test"]

best_without = {
    "config": {
        "type": "without",
        "cn_train_ratio": 0.5,                 # 如你改了比例，记得同步
        "use_ci_in_train": False,
        "seed": 123,
    },
    "data_split": {
        "train_cn_ids": train_cn_df["anon_id"].tolist() if "anon_id" in train_cn_df.columns else [],
        "test": [
            {
                "anon_id": row["anon_id"] if "anon_id" in test_df.columns else int(i),
                "label": row["label"] if "label" in test_df.columns else None,
                "label_binary": int(row["label_binary"]) if "label_binary" in test_df.columns else None,
            }
            for i, row in test_df.reset_index(drop=True).iterrows()
        ],
    },
    "decision": {
        "weights_name": str(_best_row["weights_name"]),
        "weights": list(map(float, _best_row["weights"])),
        "threshold": float(th_wo),
        "center": list(map(float, _ctx["center"])),
        "std": list(map(float, _ctx["std"])),
    },
    "score": float(_best_row["f1"]),  # 你之前用F1作为主score
    "metrics": {
        "scan": {
            "f1": float(_best_row["f1"]),
            "accuracy": float(_best_row["acc"]),
            "precision": float(_best_row["prec"]),
            "recall": float(_best_row["rec"]),
            "p_value_mannwhitney": float(_best_row["p_value"]) if pd.notnull(_best_row["p_value"]) else None,
            "separation_mean_ci_minus_cn": float(_best_row["sep"]),
        },
        "final": metrics_wo  # 基于最终阈值的混淆矩阵等
    }
}

json_dump_safe(best_without, "best_without_config.json")
print("✓ Saved best_without_config.json")



# ---------------- best WITH (fixed_fpr 无标签) ----------------
# 重要：把第三个返回值接出来，拿到 per_weight 阈值与全局回退
pers_z_summary, pers_z_detail, with_ctx = evaluate_personalized(
    df_all, train_ratio=0.5, seed=123, method="fixed_fpr", alpha=0.975
)

y_true_w  = pers_z_detail["true"].values
y_pred_w  = pers_z_detail["pred"].values
metrics_w = compute_metrics_rich(y_true_w, y_pred_w)

# 每个权重的阈值字典（便于复现）
per_weight = with_ctx["per_weight"]           # {name: {"w":..., "d_test":..., "th":...}}
thresholds_per_weight = {
    name: float(obj["th"]) for name, obj in per_weight.items()
}
weights_vectors = {
    name: list(map(float, obj["w"])) for name, obj in per_weight.items()
}

# 逐人条目：chosen_weight / 距离 / 阈值 / margin / 真值/预测
# pers_z_detail 里已含 chosen_weight、margin、true、pred
# 距离与阈值需要从 per_weight 或 gf 中取
gf = with_ctx.get("global_fallback", None)

per_subject = []
# 需要与 test 对齐 anon_id/label
test_df_w = with_ctx["test"].reset_index(drop=True)
for i, row in pers_z_detail.reset_index(drop=True).iterrows():
    chosen_name = row["chosen_weight"]
    if chosen_name == "GLOBAL_FALLBACK" and gf is not None:
        # 全局回退：距离/阈值来自gf
        dist_i = float(gf["d"][i])
        th_i   = float(gf["th"])
        w_vec  = None  # 全局已在 without JSON 里给出
    else:
        dist_i = float(per_weight[chosen_name]["d_test"][i])
        th_i   = float(per_weight[chosen_name]["th"])
        w_vec  = weights_vectors.get(chosen_name, None)

    per_subject.append({
        "anon_id": test_df_w["anon_id"].iloc[i] if "anon_id" in test_df_w.columns else int(i),
        "label": test_df_w["label"].iloc[i] if "label" in test_df_w.columns else None,
        "true": int(row["true"]),
        "pred": int(row["pred"]),
        "chosen_weight_name": chosen_name,
        "chosen_weight_vector": w_vec,    # GLOBAL_FALLBACK 为 None
        "threshold_used": th_i,
        "distance": dist_i,
        "margin": float(row["margin"]),
    })

# 统计最常被选的权重
weights_top_counts = pers_z_summary.get("weights_top_counts", {})

best_with = {
    "config": {
        "type": "with",
        "selection": "fixed_fpr",
        "alpha": float(pers_z_summary.get("alpha", 0.975)),
        "cn_train_ratio": 0.5,
        "use_ci_in_train": False,
        "seed": 123,
        "global_fallback_used": gf is not None,
    },
    "data_split": {
        "train_cn_ids": with_ctx["train_cn"]["anon_id"].tolist() if "anon_id" in with_ctx["train_cn"].columns else [],
        "test": [
            {
                "anon_id": r["anon_id"] if "anon_id" in with_ctx["test"].columns else int(i),
                "label": r["label"] if "label" in with_ctx["test"].columns else None,
                "label_binary": int(r["label_binary"]) if "label_binary" in with_ctx["test"].columns else None,
            }
            for i, r in with_ctx["test"].reset_index(drop=True).iterrows()
        ],
    },
    "thresholds_per_weight": thresholds_per_weight,  # 每个权重对应的CN分位阈值
    "weights_library": weights_vectors,              # 每个权重的向量（便于复现）
    "per_subject": per_subject,                      # ★ 逐人设置（你关心的）
    "score": float(pers_z_summary["f1"]),
    "metrics": {
        "summary": {
            "f1": float(pers_z_summary["f1"]),
            "accuracy": float(pers_z_summary["acc"]),
            "precision": float(pers_z_summary["prec"]),
            "recall": float(pers_z_summary["rec"]),
            "p_value_mannwhitney": float(pers_z_summary["p_value"]) if pd.notnull(pers_z_summary["p_value"]) else None,
            "weights_top_counts": weights_top_counts,
        },
        "final": metrics_w
    }
}

json_dump_safe(best_with, "best_with_config.json")  # ← 确保这行存在
print("✓ Saved best_with_config.json")  

json_dump_safe({"best_without": best_without, "best_with": best_with}, "best_configs_summary.json")
print("✓ Saved best_configs_summary.json")

# ---------- 立即校验 ----------
for p in ["best_without_config.json","best_with_config.json","best_configs_summary.json"]:
    print(f"[check] {p}: exists={os.path.exists(p)}, size={os.path.getsize(p) if os.path.exists(p) else 0} bytes")

# 控制台简表（方便拷论文）
def short_row(tag, m):
    return {
        "Mode": tag,
        "ACC": f"{m['accuracy']:.3f}",
        "F1": f"{m['f1']:.3f}",
        "Precision": f"{m['precision']:.3f}",
        "Recall/Sens.": f"{m['recall_sensitivity']:.3f}",
        "Specificity": f"{m['specificity']:.3f}",
        "Bal.ACC": f"{m['balanced_accuracy']:.3f}",
        "MCC": f"{m['mcc']:.3f}",
    }

print("\n=== Summary (for paper) ===")
print(pd.DataFrame([
    short_row("Without (global)", best_without["metrics"]["final"]),
    short_row("With (personalized)", best_with["metrics"]["final"]),
]).to_string(index=False))



=== WITHOUT personalization（全局最优）===
         weights_name                                            weights  \
0          Movement++                               [0.1, 0.1, 0.7, 0.1]   
1  Circadian-dominant                             [0.7, 0.2, 0.05, 0.05]   
2               Equal                           [0.25, 0.25, 0.25, 0.25]   
3            Social++                               [0.1, 0.1, 0.1, 0.7]   
4    Grid_C0.50_T0.35  [0.5, 0.35, 0.07500000000000001, 0.07500000000...   
5    Grid_C0.45_T0.40  [0.45, 0.39999999999999997, 0.0750000000000000...   
6               CT+++                             [0.5, 0.4, 0.05, 0.05]   
7    Grid_C0.50_T0.40  [0.5, 0.39999999999999997, 0.05000000000000004...   
8         Circadian++                             [0.5, 0.35, 0.1, 0.05]   
9         CT-Balanced                           [0.45, 0.45, 0.05, 0.05]   

   threshold        f1       acc      prec      rec   p_value       sep  
0   0.074354  0.854167  0.745455  0.759259  0.97619